> lingo：行话，术语；

- github:
    - https://github.com/Huanshere/VideoLingo
- 官方文档：
    - https://videolingo.io/docs/introduction
- https://colab.research.google.com/github/Huanshere/VideoLingo/blob/main/VideoLingo_colab.ipynb
- requirements
    - WhisperX（本地 whisper）

- install
    ```
    git clone https://github.com/Huanshere/VideoLingo.git
    cd VideoLingo
    
    conda create -n video python=3.12
    conda activate video
    
    # 注意这里的安装是交互式，可能需要一些手动输入的参与（或者获取 sudo 权限）
    python install 1

    # 启动web服务
    streamlit run st.py
    ```
- docker
```
docker pull rqlove/videolingo:latest
docker run -d -p 8501:8501 --gpus all rqlove/videolingo:latest
```
- steps
    - Download or Upload Video
        - 这一步执行完成的标志即是能页面播放视频时；
    - Translate and Generate Subtitles
    - Dubbing
- 界面
    - Vocal Separation Enhance
        - Recommended for videos with loud background noise, but will increase processing time
    - Burn-in subtiles：takes longer time
- 其他
    - 界面上的配置都是加载的代码根目录的 `config.yaml`
    - 如果上传的是纯 audio，则会用 ffmpeg 在 output 文件夹首先生成 audio_with_black_screen.mp4 的视频文件；
    - 划 segments：600s（10mins）为一个chunk
    - 所有的过程文件（比如字幕文件）都在 output 文件夹下
        - Include Video：output_video_with_subs.mp4;

### UVR => Demucs

Ultimate Vocal Remover (UVR) 工具通常可以用于以下任务：

- 去除人声：从歌曲中移除人声，保留伴奏（俗称“卡拉OK”处理）。
- 提取人声：从混音中分离并提取出人声，保留干净的人声音轨。

当有较大背景音乐时，如果不进行人声分离（vocal separation），可能会导致单词级字幕识别不准确，从而在最后的对齐步骤出现错误。

raw_full_audio.wav
- original_vocal.wav
- background.wav

misc
- de-echo：去回声；

- demucs
    - raw.mp3 => background.mp3

### ASR

- asr
    - chinese: BELLE-2/Belle-whisper-large-v3-zh-punct
        - Belle-whisper-large-v3-zh + punctuation mark 
    - english: large-v3

### whisperx

### transformers

```
import torch
from transformers import pipeline

transcriber = pipeline(
  "automatic-speech-recognition", 
  model="BELLE-2/Belle-whisper-large-v3-zh-punct",
  device="cuda" if torch.cuda.is_available() else "cpu"
)

transcriber.model.config.forced_decoder_ids = (
  transcriber.tokenizer.get_decoder_prompt_ids(
    language="zh", 
    task="transcribe"
  )
)

transcription = transcriber("my_audio.wav")
```

### Dubbing

- 指将音频中的原始语音或声音替换为新的音轨。

### LLM

- base url: https://api.openai.com/
- model: gpt-40-min

#### sentence split by meaning

In [3]:
s = "### Role\nYou are a professional and experienced Netflix subtitle splitter in zh.\n\n### Task\nYour task is to split the given subtitle text into **2** parts, each should be less than 20 words.\n\n### Requirements\n1. Try to maintain the coherence of the sentence meaning, split according to Netflix subtitle standards, ensuring the two parts are relatively independent.\n2. The length of each part should be roughly equal, no part should be less than 3 words, but the integrity of the sentence is more important.\n3. Prioritize splitting at punctuation marks, such as periods, commas, and conjunctions (e.g., \"and\", \"but\", \"because\", \"when\", \"then\", \"if\", \"so\", \"that\").\n\n### Steps\n1. Analyze the grammar and structure of the given text.\n2. Provide 2 different ways to split the text, each with different split points, output complete sentences (do not change any letters or punctuation), insert [br] tags at the split positions.\n3. Briefly compare and evaluate the above 2 split methods, considering readability, grammatical structure, and contextual coherence, choose the best split method.\n4. Give the best split method number, 1 or 2.\n\n### Output Format\nPlease provide your answer in the following JSON format, <<>> represents placeholders:\n{\n    \"analysis\": \"Brief analysis of the text structure and split strategy\",\n    \"split_1\": \"<<The first split method, output complete sentences, insert [br] as a delimiter at the split position. e.g. this is the first part [br] this is the second part.>>\",\n    \"split_2\": \"<<The second split method>>\",\n    \"eval\": \"<<Unified brief evaluation of the 2 split methods, written in one sentence, no line breaks>>\",\n    \"best\": \"<<The best split method number, 1 or 2>>\"\n}\n\n### Given Text\n<split_this_sentence>\n个概念都会用到就是Mt-S就是Megatransferspersecond就是每秒传输的大M的这个表示的克瓦特传输的次数我们可以用Mt来衡量再除以1024就是用Gt来衡量\n</split_this_sentence>"

In [4]:
print(s)

### Role
You are a professional and experienced Netflix subtitle splitter in zh.

### Task
Your task is to split the given subtitle text into **2** parts, each should be less than 20 words.

### Requirements
1. Try to maintain the coherence of the sentence meaning, split according to Netflix subtitle standards, ensuring the two parts are relatively independent.
2. The length of each part should be roughly equal, no part should be less than 3 words, but the integrity of the sentence is more important.
3. Prioritize splitting at punctuation marks, such as periods, commas, and conjunctions (e.g., "and", "but", "because", "when", "then", "if", "so", "that").

### Steps
1. Analyze the grammar and structure of the given text.
2. Provide 2 different ways to split the text, each with different split points, output complete sentences (do not change any letters or punctuation), insert [br] tags at the split positions.
3. Briefly compare and evaluate the above 2 split methods, considering readabi

#### subtile trim

In [5]:
s = "\n### Role\nYou are a professional subtitle editor, editing and optimizing lengthy subtitles that exceed voiceover time before handing them to voice actors. Your expertise lies in cleverly shortening subtitles slightly while ensuring the original meaning and structure remain unchanged.\n\n### Subtitle Data\n<subtitles>\nSubtitle: \"希望那一天能尽快到来，以上就是本期的全部内容。\"\nDuration: 3.2719999999999914 seconds\n</subtitles>\n\n### Processing Rules\nConsider a. Reducing filler words without modifying meaningful content. b. Omitting unnecessary modifiers or pronouns, for example:\n    - \"Please explain your thought process\" can be shortened to \"Please explain thought process\"\n    - \"We need to carefully analyze this complex problem\" can be shortened to \"We need to analyze this problem\"\n    - \"Let's discuss the various different perspectives on this topic\" can be shortened to \"Let's discuss different perspectives on this topic\"\n    - \"Can you describe in detail your experience from yesterday\" can be shortened to \"Can you describe yesterday's experience\" \n\n### Processing Steps\nPlease follow these steps and provide the results in the JSON output:\n1. Analysis: Briefly analyze the subtitle's structure, key information, and filler words that can be omitted.\n2. Trimming: Based on the rules and analysis, optimize the subtitle by making it more concise according to the processing rules.\n\n### Output Format\nPlease complete the following JSON data, where << >> represents content you need to fill in:\n{\n    \"analysis\": \"<<Brief analysis of the subtitle, including structure, key information, and potential processing locations>>\",\n    \"trans_text_processed\": \"<<Optimized and shortened subtitle in the original subtitle language>>\"\n}\n"

In [6]:
print(s)


### Role
You are a professional subtitle editor, editing and optimizing lengthy subtitles that exceed voiceover time before handing them to voice actors. Your expertise lies in cleverly shortening subtitles slightly while ensuring the original meaning and structure remain unchanged.

### Subtitle Data
<subtitles>
Subtitle: "希望那一天能尽快到来，以上就是本期的全部内容。"
Duration: 3.2719999999999914 seconds
</subtitles>

### Processing Rules
Consider a. Reducing filler words without modifying meaningful content. b. Omitting unnecessary modifiers or pronouns, for example:
    - "Please explain your thought process" can be shortened to "Please explain thought process"
    - "We need to carefully analyze this complex problem" can be shortened to "We need to analyze this problem"
    - "Let's discuss the various different perspectives on this topic" can be shortened to "Let's discuss different perspectives on this topic"
    - "Can you describe in detail your experience from yesterday" can be shortened to "Can 

#### summary

In [10]:
s = "### Role\nYou are a professional video translation expert and terminology consultant. Your expertise lies not only in accurately understanding the original zh text but also in extracting key professional terms and optimizing the translation to better suit the expression habits and cultural background of Chinese.\n\n### Task Description \nFor the provided original zh video text, you need to:\n1. Summarize the video's main topic in one sentence\n2. Extract professional terms and names that appear in the video, and provide Chinese translations or suggest keeping the original language terms. Avoid extracting simple, common words.\n3. For each translated term or name, provide a brief explanation\n\n### Analysis and Summary Steps\nPlease think in two steps, processing the text line by line:  \n1. Topic summarization:\n   - Quickly skim through the entire text to understand the general idea\n   - Summarize the topic in one concise sentence\n2. Term and name extraction:\n   - Carefully read the entire text, marking professional terms and names\n   - For each term or name, provide a Chinese translation or suggest keeping the original, only the word itself is needed, not the pronunciation\n   - Add a brief explanation for each term or name to help the translator understand\n   - If the word is a fixed abbreviation or a proper name, please keep the original.\n\n### Output Format\nPlease output your analysis results in the following JSON format, where <> represents placeholders:\n{\n    \"theme\": \"<Briefly summarize the theme of this video in 1 sentence>\",\n    \"terms\": [\n        {\n            \"original\": \"<Term or name 1 in the zh>\",\n            \"translation\": \"<Chinese translation or keep original>\",\n            \"explanation\": \"<Brief explanation of the term or name>\"\n        },\n        {\n            \"original\": \"<Term or name 2 in the zh>\",\n            \"translation\": \"<Chinese translation or keep original>\",\n            \"explanation\": \"<Brief explanation of the term or name>\"\n        },\n        ...\n    ]\n}\n\n### Single Output Example (Using French as an example)\n\n{\n    \"theme\": \"Ce vidéo résume le musée du Louvre à Paris.\",\n    \"terms\": [\n        {\n            \"original\": \"Mona Lisa\",\n            \"translation\": \"La Joconde\",\n            \"explanation\": \"Le tableau le plus célèbre du Louvre, un portrait de Léonard de Vinci\"\n        },\n        {\n            \"original\": \"pyramid\",\n            \"translation\": \"la pyramide\",\n            \"explanation\": \"Une grande structure en verre et métal en forme de pyramide située à l'entrée principale du Louvre\"\n        },\n        {\n            \"original\": \"I.M. Pei\",\n            \"translation\": \"I.M. Pei\",\n            \"explanation\": \"L'architecte américain d'origine chinoise qui a conçu la pyramide du Louvre\"\n        },\n        ...\n    ]\n}\n\n### Video text data to be processed\n<video_text_to_summarize>\n各 位 朋 友 们 大 家 周 末 好 今 天 我 们 继 续 回 到 小 白 象 深 度 学 习 装 机 指 南...\n</video_text_to_summarize>"

In [11]:
print(s)

### Role
You are a professional video translation expert and terminology consultant. Your expertise lies not only in accurately understanding the original zh text but also in extracting key professional terms and optimizing the translation to better suit the expression habits and cultural background of Chinese.

### Task Description 
For the provided original zh video text, you need to:
1. Summarize the video's main topic in one sentence
2. Extract professional terms and names that appear in the video, and provide Chinese translations or suggest keeping the original language terms. Avoid extracting simple, common words.
3. For each translated term or name, provide a brief explanation

### Analysis and Summary Steps
Please think in two steps, processing the text line by line:  
1. Topic summarization:
   - Quickly skim through the entire text to understand the general idea
   - Summarize the topic in one concise sentence
2. Term and name extraction:
   - Carefully read the entire text, 

#### translate_expressiveness

In [12]:
s = "### Role Definition\nYou are a professional Netflix subtitle translator and language consultant. Your expertise lies not only in accurately understanding the original zh but also in optimizing the Chinese translation to better suit the target language's expression habits and cultural background.\n\n### Task Background\nWe already have a direct translation version of the original zh subtitles. Now we need you to reflect on and improve these direct translations to create more natural and fluent Chinese subtitles.\n\n### Task Description\nBased on the provided original zh text and Chinese direct translation, you need to:\n1. Analyze the direct translation results line by line, pointing out existing issues\n2. Provide detailed modification suggestions\n3. Perform free translation based on your analysis\n\n### Context Information\n<previous_content>\n['然后A100的话我们再来看这个A100就是A100的话它是5120bit', '它的位宽是非常高的它是一个数量级的一个差距它是5120位它的', '显存类型就不再是DDR这个系列了而是HBM就是高带宽的一个Memory']\n</previous_content>\n\n<subsequent_content>\n['也是得到了128位内存带宽的计算的话就是这个地方就是频率乘以你这个位宽', '再除以8']\n</subsequent_content>\n\n### Content Summary\n本视频讨论了内存带宽在深度学习和人工智能领域中的重要性，并对比了英伟达和苹果M4系列芯片的性能。\n\n### Points to Note\n3. \"M4系列\": \"M4系列\", meaning: 苹果近期发布的一系列高性能芯片，专注于移动设备和计算任务，尤其在神经网络运算方面有突出的表现。\n4. \"显存\": \"显存\", meaning: 显卡专用的内存，用于存储图像数据和其他相关信息，以支持实时图形处理。\n\n### Translation Analysis Steps\nPlease use a two-step thinking process to handle the text line by line:\n\n1. Direct Translation Reflection:\n   - Evaluate language fluency\n   - Check if the language style is consistent with the original text\n   - Check the conciseness of the subtitles, point out where the translation is too wordy, the translation should be close to the original text in length\n\n2. Chinese Free Translation:\n   - Based on the reflection in step 1, perform free translation\n   - Aim for contextual smoothness and naturalness, conforming to Chinese expression habits\n   - Ensure it's easy for Chinese audience to understand and accept\n   - Keep the subtitles concise, with a plain and natural language style, and maintain consistency in structure between the free translation and the zh original\n\n### Subtitle Data\n<subtitles>\n其他的数据A100我就不再算了\n我们重点来看M4系列的芯片\nM4系列芯片的话它有分M4M4Pro和M4Max它们的内存的类型就是\nLPDDR5X它的显存频率它的内存频率是7500转\n然后M4Pro和M4Max它用的是8533MT-S好它的内存位宽\n的话M4的内存位宽是64比特乘以2\n应该是两个通道这个2的含义\n我不是具体特别清楚它最终种子\n它的内存位宽是128位当然VK里边它又介绍\n了这是一个我们看M4的话它这里边有一个8乘以16应该是8乘以16\n</subtitles>\n\n### Output Format\nMake sure to generate the correct Json format, don't output \" in the value.\nPlease complete the following JSON data, where << >> represents placeholders that should not appear in your answer, and return your translation results in JSON format:\n{\n    \"1\": {\n        \"origin\": \"其他的数据A100我就不再算了\",\n        \"direct\": \"我就不再考虑其他的数据A100了\",\n        \"reflection\": \"<<reflection on the direct translation version>>\",\n        \"free\": \"<<retranslated result, aiming for fluency and naturalness, conforming to Chinese expression habits, DO NOT leave empty line here!>>\"\n    },\n    \"2\": {\n        \"origin\": \"我们重点来看M4系列的芯片\",\n        \"direct\": \"让我们重点关注M4系列芯片\",\n        \"reflection\": \"<<reflection on the direct translation version>>\",\n        \"free\": \"<<retranslated result, aiming for fluency and naturalness, conforming to Chinese expression habits, DO NOT leave empty line here!>>\"\n    },\n    \"3\": {\n        \"origin\": \"M4系列芯片的话它有分M4M4Pro和M4Max它们的内存的类型就是\",\n        \"direct\": \"M4系列芯片分为M4、M4 Pro和M4 Max，它们的内存类型是\",\n        \"reflection\": \"<<reflection on the direct translation version>>\",\n        \"free\": \"<<retranslated result, aiming for fluency and naturalness, conforming to Chinese expression habits, DO NOT leave empty line here!>>\"\n    },\n    \"4\": {\n        \"origin\": \"LPDDR5X它的显存频率它的内存频率是7500转\",\n        \"direct\": \"LPDDR5X，其显存频率和内存频率为7500MHz\",\n        \"reflection\": \"<<reflection on the direct translation version>>\",\n        \"free\": \"<<retranslated result, aiming for fluency and naturalness, conforming to Chinese expression habits, DO NOT leave empty line here!>>\"\n    },\n    \"5\": {\n        \"origin\": \"然后M4Pro和M4Max它用的是8533MT-S好它的内存位宽\",\n        \"direct\": \"而M4 Pro和M4 Max采用的是8533MT-S，其内存位宽\",\n        \"reflection\": \"<<reflection on the direct translation version>>\",\n        \"free\": \"<<retranslated result, aiming for fluency and naturalness, conforming to Chinese expression habits, DO NOT leave empty line here!>>\"\n    },\n    \"6\": {\n        \"origin\": \"的话M4的内存位宽是64比特乘以2\",\n        \"direct\": \"则M4的内存位宽为64位乘以2\",\n        \"reflection\": \"<<reflection on the direct translation version>>\",\n        \"free\": \"<<retranslated result, aiming for fluency and naturalness, conforming to Chinese expression habits, DO NOT leave empty line here!>>\"\n    },\n    \"7\": {\n        \"origin\": \"应该是两个通道这个2的含义\",\n        \"direct\": \"这意味着有两个通道，这是2的含义\",\n        \"reflection\": \"<<reflection on the direct translation version>>\",\n        \"free\": \"<<retranslated result, aiming for fluency and naturalness, conforming to Chinese expression habits, DO NOT leave empty line here!>>\"\n    },\n    \"8\": {\n        \"origin\": \"我不是具体特别清楚它最终种子\",\n        \"direct\": \"我不太清楚它的最终配置\",\n        \"reflection\": \"<<reflection on the direct translation version>>\",\n        \"free\": \"<<retranslated result, aiming for fluency and naturalness, conforming to Chinese expression habits, DO NOT leave empty line here!>>\"\n    },\n    \"9\": {\n        \"origin\": \"它的内存位宽是128位当然VK里边它又介绍\",\n        \"direct\": \"它的内存位宽为128位，VK中也提到过\",\n        \"reflection\": \"<<reflection on the direct translation version>>\",\n        \"free\": \"<<retranslated result, aiming for fluency and naturalness, conforming to Chinese expression habits, DO NOT leave empty line here!>>\"\n    },\n    \"10\": {\n        \"origin\": \"了这是一个我们看M4的话它这里边有一个8乘以16应该是8乘以16\",\n        \"direct\": \"这是一个，我们看M4，它的内部有8乘以16，应该是8乘以16\",\n        \"reflection\": \"<<reflection on the direct translation version>>\",\n        \"free\": \"<<retranslated result, aiming for fluency and naturalness, conforming to Chinese expression habits, DO NOT leave empty line here!>>\"\n    }\n}"

In [13]:
print(s)

### Role Definition
You are a professional Netflix subtitle translator and language consultant. Your expertise lies not only in accurately understanding the original zh but also in optimizing the Chinese translation to better suit the target language's expression habits and cultural background.

### Task Background
We already have a direct translation version of the original zh subtitles. Now we need you to reflect on and improve these direct translations to create more natural and fluent Chinese subtitles.

### Task Description
Based on the provided original zh text and Chinese direct translation, you need to:
1. Analyze the direct translation results line by line, pointing out existing issues
2. Provide detailed modification suggestions
3. Perform free translation based on your analysis

### Context Information
<previous_content>
['然后A100的话我们再来看这个A100就是A100的话它是5120bit', '它的位宽是非常高的它是一个数量级的一个差距它是5120位它的', '显存类型就不再是DDR这个系列了而是HBM就是高带宽的一个Memory']
</previous_content>

<subsequent_content>


#### translate_faithfulness

In [14]:
s = "### Role Definition\nYou are a professional Netflix subtitle translator, fluent in both zh and Chinese, as well as their respective cultures. Your expertise lies in accurately understanding the semantics and structure of the original zh text and faithfully translating it into Chinese while preserving the original meaning.\n\n### Task Background\nWe have a segment of original zh subtitles that need to be directly translated into Chinese. These subtitles come from a specific context and may contain specific themes and terminology.\n\n### Task Description\nBased on the provided original zh subtitles, you need to:\n1. Translate the original zh subtitles into Chinese line by line\n2. Ensure the translation is faithful to the original, accurately conveying the original meaning\n3. Consider the context and professional terminology\n\n### Context Information\n<previous_content>\n['我不是具体特别清楚它最终种子', '它的内存位宽是128位当然VK里边它又介绍', '了这是一个我们看M4的话它这里边有一个8乘以16应该是8乘以16']\n</previous_content>\n\n<subsequent_content>\n['只是它的算术更快它的频率更', '快8533大概提升了1000好']\n</subsequent_content>\n\n### Content Summary\n本视频讨论了内存带宽在深度学习和人工智能领域中的重要性，并对比了英伟达和苹果M4系列芯片的性能。\n\n### Points to Note\n1. \"内存带宽\": \"内存带宽\", meaning: 指计算机中内存能够传输数据的速率，通常以每秒传输的数据量（如GB/s或bits/s）来衡量。\n\n### Translation Principles\n1. Faithful to the original: Accurately convey the content and meaning of the original text, without arbitrarily changing, adding, or omitting content.\n2. Accurate terminology: Use professional terms correctly and maintain consistency in terminology.\n3. Understand the context: Fully comprehend and reflect the background and contextual relationships of the text.\n\n### Subtitle Data\n<subtitles>\n也是得到了128位内存带宽的计算的话就是这个地方就是频率乘以你这个位宽\n再除以8\n因为目前还是一个m然后再除以1000就变成了一个gb每秒就120gb每秒\n我们看这个数据也是对的120就是8乘以16表示它\n这个controllers这个我不知道什么含义它的意思就是8乘以16\n就是每一个内存的一个位宽是16比特\n一共8个这样的controllers就16×8是\n128个比特位宽然后最终算下来的话就是120GB每秒\n然后对于MSPro和MSMax它用到的是同样的\n我们看它用到的是同一个内存的一个类型LPDDR5X\n</subtitles>\n\n### Output Format\nPlease complete the following JSON data, where << >> represents placeholders that should not appear in your answer, and return your translation results in JSON format:\n{\n    \"1\": {\n        \"origin\": \"也是得到了128位内存带宽的计算的话就是这个地方就是频率乘以你这个位宽\",\n        \"direct\": \"<<direct Chinese translation>>\"\n    },\n    \"2\": {\n        \"origin\": \"再除以8\",\n        \"direct\": \"<<direct Chinese translation>>\"\n    },\n    \"3\": {\n        \"origin\": \"因为目前还是一个m然后再除以1000就变成了一个gb每秒就120gb每秒\",\n        \"direct\": \"<<direct Chinese translation>>\"\n    },\n    \"4\": {\n        \"origin\": \"我们看这个数据也是对的120就是8乘以16表示它\",\n        \"direct\": \"<<direct Chinese translation>>\"\n    },\n    \"5\": {\n        \"origin\": \"这个controllers这个我不知道什么含义它的意思就是8乘以16\",\n        \"direct\": \"<<direct Chinese translation>>\"\n    },\n    \"6\": {\n        \"origin\": \"就是每一个内存的一个位宽是16比特\",\n        \"direct\": \"<<direct Chinese translation>>\"\n    },\n    \"7\": {\n        \"origin\": \"一共8个这样的controllers就16×8是\",\n        \"direct\": \"<<direct Chinese translation>>\"\n    },\n    \"8\": {\n        \"origin\": \"128个比特位宽然后最终算下来的话就是120GB每秒\",\n        \"direct\": \"<<direct Chinese translation>>\"\n    },\n    \"9\": {\n        \"origin\": \"然后对于MSPro和MSMax它用到的是同样的\",\n        \"direct\": \"<<direct Chinese translation>>\"\n    },\n    \"10\": {\n        \"origin\": \"我们看它用到的是同一个内存的一个类型LPDDR5X\",\n        \"direct\": \"<<direct Chinese translation>>\"\n    }\n}"

In [15]:
print(s)

### Role Definition
You are a professional Netflix subtitle translator, fluent in both zh and Chinese, as well as their respective cultures. Your expertise lies in accurately understanding the semantics and structure of the original zh text and faithfully translating it into Chinese while preserving the original meaning.

### Task Background
We have a segment of original zh subtitles that need to be directly translated into Chinese. These subtitles come from a specific context and may contain specific themes and terminology.

### Task Description
Based on the provided original zh subtitles, you need to:
1. Translate the original zh subtitles into Chinese line by line
2. Ensure the translation is faithful to the original, accurately conveying the original meaning
3. Consider the context and professional terminology

### Context Information
<previous_content>
['我不是具体特别清楚它最终种子', '它的内存位宽是128位当然VK里边它又介绍', '了这是一个我们看M4的话它这里边有一个8乘以16应该是8乘以16']
</previous_content>

<subsequent_content>
['只是它的算术